## 波次优化(BatchOpt)

介绍波次优化问题,建立数学模型以及给出一些模型改进方式,并调用Gurobi求解模型。

### 问题描述

电商等仓储物流中心每天需要配送几万甚至十几万包裹,每个包裹需要包含来自于不同仓库,不同库区,不同数量的物品。如果规划不合理,可能会导致跨库、跨区域数量过多,订单配货时间过长,物流核心设备无效冗余运作距离过长,库存周转效率降低等问题。

例如某电商有$P$个包裹(每个包裹含多件商品,可能分布在多个仓库(暂时不考虑不同库区))需将其分到$B$个波次中,要求：

1. 每个波次的商品件数在$[G1,G2]$件之间;
2. 每个波次的包裹数量在$[P1,P2]$之间;
3. 波次包含的不同的仓库数量最少。

- **输入数据格式**

1. `pakage_no`：包裹号;
2. `warehouse`：仓库;
3. `goods_qty`：在仓库中的商品件数。



|pakage_no|warehouse|goods_qty|
|:-|:-|:-|
|302826075|2号仓-13|2|
|302826075|2号仓-23|1|
|302826075|5号仓-3|1|
|311001535|2号仓-24|1|
|311001535|佳明仓-3|1|
|311308096|2号仓-11|3|

### Version1

&emsp;&emsp;**最简单的思路构建模型(version1)**

1. **参数**
    - Set $I$ ~ 包裹集合
    - Set $J$ ~ 波次集合
    - Set $K$ ~ 仓库集合
    - $B$ ~ 波次总数
    - $M$ ~ 足够大的数
    - $Q_i$ ~ 包裹$i$的商品件数
    - $c_{ik}$ ~ 包裹$i$是否有仓库$k$的商品
    - $G_1, G_2$ ~ 单一波次商品件数限制
    - $P_1, P_2$ ~ 单一波次包裹数量限制
    
2. **变量**

    - $x_{ij}$ ~ binary, 包裹$i$是否属于波次$j$。
    - $y_{jk}$ ~ binary, 波次$j$是否用到仓库$k$。
    - $z_j$ ~ binary, 波次$j$是否被用到。
    
3. **model**:

$$
min \ \sum_{j \in J} \sum_{k \in K} y_{jk} \\
s.t. \ \sum_{j \in J} z_j = B \\
\ \ \sum_{j \in J} x_{ij} = 1, \forall i \in I \\
G_1 z_j \le \sum_{i \in I} Q_i x_{ij} \le G_2 z_j, \forall j \in J \\
P_1 z_j \le \sum_{i \in I} x_{ij} \le P_2 z_j, \forall j \in J \\
M y_{jk} \ge \sum_{i \in I} c_{ik} x_{ij}, \forall j \in j, k \in K
$$
    
3. **obj**

    - $min \ \sum_{j \in J} \sum_{k \in K} y_{jk}$ ~ 波次包含的不同仓库数量之和最小

4. **s.t.**

    - $\sum_{j \in J} z_j = B$ ~ 保证分配的波次数量
    - $\sum_{j \in J} x_{ij} = 1$, $\forall i \in I$ ~ 保证每个包裹都安排到波次中去
    - $G_1 z_j \le \sum_{i \in I} Q_i x_{ij} \le G_2 z_j$, $\forall j \in J$ ~ 保证单一波次商品件数在区间内
    - $P_1 z_j \le \sum_{i \in I} x_{ij} \le P_2 z_j$, $\forall j \in J$ ~ 保证单一波次包裹数量在区间内
    - $M y_{jk} \ge \sum_{i \in I} c_{ik} x_{ij}$, $\forall j \in j, k \in K$ ~ 确认波次$j$是否用到仓库$k$。
   

**version1的问题所在**

若电商有P个包裹需将其分配到B个波次中，且假设涉及到的仓库数量为K

变量数 = P*B + K*B + B

约束数 = P + 4B + B*K + 1

本次测试案例包裹数量57780, 波次数量107, 仓库数量4。则可以得出, 变量数为6182995, 约束数为58637

version1模型建模思路简单容易想到，但求解时间随着包裹、波次、仓库的数量增加而快速增加，求解起来非常耗时，因此该模型在实际场景下很难应用。

### Version2

**version2思路**

目标是使得所有波次包含的不同仓库数量总和最少，所以我们建模的时候可以将重点只放到仓库上，而不是包裹上。所以做一个模式(pattern)的统计, 包裹涉及到的仓库记为1, 否则为0。(例如一共四个仓库, 包裹A涉及到了前3个仓库, 则对应的pattern为{1,1,1,0}) 

用涉及仓库模式取代具体包裹，会极大降低问题规模(只考虑波次用了该模式多少次而不需要具体区分包裹)。因为问题要求单一波次的商品件数在[G1,G2]件之间, 所以需要在pattern里增加商品件数这一维度。

Pattern: {包裹商品件数, 仓库1, 仓库2, 仓库3, 仓库4}

- 参数
    - $P$ ~ 模式集合
    - $J$ ~ 波次集合
    - $K$ ~ 仓库集合
    - $B$ ~ 波次总数
    - $M$ ~ 足够大的数
    - $q_p$ ~ 模式p的包裹的数量
    - $Q_p$ ~ 模式p的商品件数
    - $c_{pk}$ ~ 模式p是否涉及到仓库k
    - $G_1, G_2$ ~ 单一波次商品件数限制
    - $P_1, P_2$ ~ 单一波次包裹数量限制
- 变量
    - $x_{pj}$ ~ integer, 波次j使用模式p的次数
    - $y_{jk}$ ~ binary, 波次j是否用到仓库k
    - $z_j$ ~ binary, 波次j是否被用到
    
- obj
    - $min \ \sum_{j \in J} \sum_{k \in K} y_{jk}$ ~ 波次包含的不同仓库数量之和最小
- s.t.
    - $\sum_{j \in J} z_j = B$ ~ 保证分配的波次数量
    - $\sum_{j \in J} x_{pj} = q_p$, $\forall i \in I$ ~ 保证模式使用次数和其包裹数量相匹配
    - $G_1 z_j \le \sum_{i \in I} Q_p x_{pj} \le G_2 z_j$, $\forall j \in J$ ~ 保证单一波次商品件数在区间内
    - $P_1 z_j \le \sum_{i \in I} x_{pj} \le P_2 z_j$, $\forall j \in J$ ~ 保证单一波次包裹数量在区间内
    - $M y_{jk} \ge \sum_{i \in I} c_{pk} x_{pj}$, $\forall j \in j, k \in K$ ~ 确认波次j是否用到仓库k


version2的变量数为11877, 约束数为963, 远小于version1，降低了问题的规模，减少了模型的求解时间

### Version3

现在开始要考虑库区，即波次包含的不同的库区数量最小。

使用version2相同的思路进行建模，如果按照库区构建Pattern。会面临与version1类似的问题，及模型规模会随着包裹和波次的数量增加而快速增加，所以需要寻找一种相对更快地处理方式

测试案例包含库区数量为22, 按照version2的思路进行建模，pattern的数量为24927， 变量数为2669650， 约束数为27710

### Version4

发现可以先判断是否属于同一仓库，若属于不同仓库，那么包裹一定属于不同库区。因此将问题分为两个阶段(stages)进行处理

阶段1: 按仓库确认每个波次使用模式(按仓库pattern)的数量, 即原来的version2
阶段2: 在第一阶段的基础上, 大幅度缩减模型规模, 进而求解获得最终结果
    
- stage1
    - 直接求解version2得到结果
- stage2
    - 通过stage1的计算，可以获得version4

- 参数
    - $\bar{J}$ ~ 用到的波次集合
    - $\bar{J}_p$ ~ 用到的波次使用pattern(仓库)集合
    - $I_p$ ~ 模式p包含的包裹集合
    - $K$ ~ 库区集合
    - $c_{ik}$ ~ 包裹i是否有区域k的商品
    - $\bar{x}_{pj}$ ~ 波次j使用模式p的次数
    - $M$ ~ 足够大的数
- 变量
    - $x_{ij}$ ~ binary, 包裹i是否属于波次j
    - $y_{jk}$ ~ binary, 波次j是否用到区域k

- obj
    - $min \ \sum_{j \in J} \sum_{k \in K} y_{jk}$ ~ 波次包含的不同库区数量总和最小
- s.t.
    - $\sum_{i \in I_p} x_{ij} = \bar{x}_{pj}$, $\forall j \in \bar{J}, p \in \bar{J}_p$ ~ 保证包裹数量要和模式使用次数匹配
    - $\sum_{j \in \bar{J}} x_{ij} = 1$, $\forall i \in I_p, p \in \bar{J}_p$ ~ 保证每个包裹都安排到波次中去
    - $M y_{jk} \ge \sum_{i \in I_p} c_{ik} x_{ij}$, $\forall j \in \bar{J}, k \in K, p \in \bar{J}_p$ ~ 确认波次j是否用到库区k

### Version5

继续改进version4, version4考虑到了具体的包裹，所以在version4的第二阶段，也可以引入pattern(库区), 进行version5的构建

- 参数
    - $\bar{J}$ ~ 用到的波次集合
    - $\bar{J}_p$ ~ 用到的波次使用pattern(仓库)集合
    - $I_p$ ~ 模式(仓库)p包含的库区模式集合
    - $K$ ~ 仓库集合
    - $q_i$ ~ 库区模式i的包裹的数量
    - $c_{ik}$ ~ 库区模式i是否涉及区域k
    - $\bar{x}_{pj}$ ~ 波次j使用模式p的次数
    - $M$ ~ 足够大的数
- 变量
    - $x_{ij}$ ~ binary, 波次j使用模式i的次数
    - $y_{jk}$ ~ binary, 波次j是否用到区域k

- obj
    - $min \ \sum_{j \in J} \sum_{k \in K} y_{jk}$ ~ 波次包含的不同库区数量总和最小
- s.t.
    - $\sum_{i \in I_p} x_{ij} = \bar{x}_{pj}$, $\forall j \in \bar{J}, p \in \bar{J}_p$ ~ 保证模式(库区)数量要和模式(仓库)使用次数匹配
    - $\sum_{j \in \bar{J}} x_{ij} = q_i$, $\forall i \in I_p, p \in \bar{J}_p$ ~ 保证模式(库区)使用量和包裹数量一致
    - $M y_{jk} \ge \sum_{i \in I_p} c_{ik} x_{ij}$, $\forall j \in \bar{J}, k \in K, p \in \bar{J}_p$ ~ 确认波次j是否用到库区k

In [1]:
from gurobipy import *
import xlrd
import xlsxwriter
from xlrd import xldate_as_tuple 
from collections import OrderedDict
import time

def ReadData(DataPath, Packages, Warehouse, Areas, Patterns, Record):
    print ("ReadData !")
    data = xlrd.open_workbook(DataPath)
    table = data.sheets()[0]
    nrows = table.nrows             #行数
    AreasCnt = 0
    WarehouseCnt = 0
    PackagesCnt = 0
    PatternsCnt = 0
    for i in range(1,nrows):
        row = table.row_values(i)   #抓取行
        Record.append(row)
        if row[1] not in Areas.keys():
            Areas[row[1]] = [AreasCnt, []]
            AreasCnt += 1
        temp = row[1].split('-')    #统计仓库
        if temp[0] not in Warehouse.keys():
            Warehouse[temp[0]] = [WarehouseCnt, []]
            WarehouseCnt += 1
        
        if row[0] in Packages.keys():
            Packages[row[0]][1] += row[2]
            if Warehouse[temp[0]][0] not in Packages[row[0]][2]: 
                Packages[row[0]][2].append(Warehouse[temp[0]][0])
                      
            if Areas[row[1]][0] not in Packages[row[0]][3]:  
                Packages[row[0]][3].append(Areas[row[1]][0])
        else:
            Packages[row[0]] = [PackagesCnt, row[2], [Warehouse[temp[0]][0]], [Areas[row[1]][0]]]
            PackagesCnt += 1
            
    WarehouseSort = list(Warehouse.keys())           
    for i in Packages.keys():
        temp = list(Packages[i][2])
        temp.sort()
        temp.insert(0, Packages[i][1])
        CheckPatterns = tuple(temp)
        if CheckPatterns in Patterns:
            Patterns[CheckPatterns].append(i)
        else:
            Patterns[CheckPatterns] = [PatternsCnt, i]
            for j in range(len(Packages[i][2])):
                Warehouse[WarehouseSort[Packages[i][2][j]]][1].append(CheckPatterns)
            PatternsCnt += 1
            
def BuildFirstStageModel(Packages, Warehouse, Patterns, Batches, PackageLimit, GoodsLimit, BatchePattern, SolveTime):
    print ("BuildModel Version 2!")
    XINDEX = {}                                                               #变量下标
    YINDEX = {}                                                               #变量下标 
    for j in range(1, Batches[1]+1):
        for i in Patterns.keys():
            XINDEX[Patterns[i][0],j] = i[0]     
        for k in range(len(Warehouse)):
            YINDEX[j,k] = 1

    model = Model()
    x = model.addVars(XINDEX.keys(), vtype=GRB.INTEGER, name='x')             #变量x_{pj}
    y = model.addVars(YINDEX.keys(), obj=YINDEX, vtype=GRB.BINARY, name='y')  #变量y_{jk} 
    z = model.addVars(Batches[1], vtype=GRB.BINARY, name='z')                 #变量z_{j}

    #约束(1) 保证分配的波次数量
    if Batches[0] == Batches[1]:
        model.addConstr(z.sum('*') == Batches[0])
    else:
        model.addConstr(z.sum('*') >= Batches[0])
        model.addConstr(z.sum('*') <= Batches[1])

    #约束(2) 保证模式使用次数和其包裹数量匹配
    for i in Patterns.keys():
        model.addConstr(x.sum(Patterns[i][0],'*') == len(Patterns[i])-1)
      
    #约束(3) 保证单一波次商品件数在区间[G1,G2]中
    for j in range(1, Batches[1]+1):
        model.addConstr(x.prod(XINDEX,'*',j) >= z[j-1]*GoodsLimit[0])
        model.addConstr(x.prod(XINDEX,'*',j) <= z[j-1]*GoodsLimit[1])
    
    #约束(4) 保证单一波次包裹数量在区间[P1,P2]中
    for j in range(1, Batches[1]+1):
        model.addConstr(x.sum('*',j) >= z[j-1]*PackageLimit[0])
        model.addConstr(x.sum('*',j) <= z[j-1]*PackageLimit[1])
    
    #约束(5) 确认波次j是否用到仓库k
    WarehouseSort = list(Warehouse.keys())  
    for j in range(1, Batches[1]+1):    
        for k in range(len(Warehouse)):
            expr = LinExpr()
            for p in range(len(Warehouse[WarehouseSort[k]][1])):
                expr += x[Patterns[Warehouse[WarehouseSort[k]][1][p]][0],j]
            model.addConstr(y[j,k] >= 0.001*expr)
    

    model.setParam(GRB.Param.TimeLimit, SolveTime)   #求解时间
    model.optimize()
    
    #获取求解结果
    for j in range(1, Batches[1]+1):
        if z[j-1].x != 0:
            for i in Patterns.keys():
                value = x[Patterns[i][0],j].x
                if value != 0:
                    if j not in BatchePattern.keys():
                        BatchePattern[j] = [[i, round(value)]]
                    else:
                        BatchePattern[j].append([i, round(value)])
                        
    
def BuildSecondStageModel(Packages, Areas, Patterns, BatchePattern, Solutions, SolveTime):
    print ("BuildModel Version 5!")
    
    XINDEX = {}                                                               #变量下标
    YINDEX = {}                                                               #变量下标
    PatternArea = {}                                                          #模式(仓库)与模式(库区)对应关系
    AreaXINDEX = {}                                                           #库区与变量下标对应关系
    for i in Patterns.keys():
        AreaPackage = OrderedDict()
        AreaPackageCnt = 0
        for j in range(1, len(Patterns[i])):
            AreaKey = Packages[Patterns[i][j]][3]
            AreaKey.sort()
            if tuple(AreaKey) not in AreaPackage.keys():
                AreaPackage[tuple(AreaKey)] = [AreaPackageCnt, Patterns[i][j]]
                AreaPackageCnt += 1
            else:
                AreaPackage[tuple(AreaKey)].append(Patterns[i][j]) 
        PatternArea[i] = AreaPackage
    
    #变量下标
    for j in BatchePattern.keys():
        for i in range(len(BatchePattern[j])):
            PatternKey = BatchePattern[j][i][0]
            for k in PatternArea[PatternKey].keys():
                XINDEX[Patterns[PatternKey][0], PatternArea[PatternKey][k][0], j] = 0              
                for kk in range(len(k)):
                    if (j,k[kk]) not in AreaXINDEX.keys():
                        AreaXINDEX[(j,k[kk])] = [(Patterns[PatternKey][0], PatternArea[PatternKey][k][0], j)]
                    else:
                        AreaXINDEX[(j,k[kk])].append((Patterns[PatternKey][0], PatternArea[PatternKey][k][0], j))                        
        for k in range(len(Areas)):
            YINDEX[j,k] = 1
            
    model = Model()
    x = model.addVars(XINDEX.keys(), vtype=GRB.INTEGER, name='x')              #变量x_{ij}
    y = model.addVars(YINDEX.keys(), obj=YINDEX, vtype=GRB.BINARY, name='y')   #变量y_{jk} 

    #约束(1) 保证模式(库区)数量要和模式(仓库)使用次数匹配
    for j in BatchePattern.keys():
        for i in range(len(BatchePattern[j])):
            PatternIndex = Patterns[BatchePattern[j][i][0]][0]
            model.addConstr(x.sum(PatternIndex, '*', j) == BatchePattern[j][i][1])
     
    #约束(2) 保证模式(库区)使用量和包裹数量一致
    for i in PatternArea.keys():
        for j in PatternArea[i].keys():
            PatternIndex = Patterns[i][0]
            PatternAreaIndex = PatternArea[i][j][0]
            model.addConstr(x.sum(PatternIndex, PatternAreaIndex, '*') == len(PatternArea[i][j])-1)
    
    #约束(3)确认波次j是否用到库区k
    for i in AreaXINDEX.keys():
        expr = LinExpr()
        for j in range(len(AreaXINDEX[i])):
            expr += x[AreaXINDEX[i][j]]
        model.addConstr(y[i] >= 0.001*expr)
            
    model.setParam(GRB.Param.TimeLimit, SolveTime)
    model.optimize()
    
    #获取求解结果
    for j in BatchePattern.keys():
        for i in range(len(BatchePattern[j])):
            PatternKey = BatchePattern[j][i][0]
            for k in PatternArea[PatternKey].keys():
                value = round(x[Patterns[PatternKey][0], PatternArea[PatternKey][k][0], j].x)
                if value != 0: 
                    for ii in range(1, value + 1):
                        Solutions[PatternArea[PatternKey][k][ii]] = j                      
                    del PatternArea[PatternKey][k][1:value + 1]
            
def OutputResult(Solutions, Record):
    workbook = xlsxwriter.Workbook('class5_Result_按库区(V2+V5).xlsx')
    worksheet = workbook.add_worksheet('方案')
    worksheet.write(0, 0, 'pakage_no')
    worksheet.write(0, 1, 'warehouse')
    worksheet.write(0, 2, 'goods_qty')
    worksheet.write(0, 3, '波次')
    
    for i in range(len(Record)):
        worksheet.write(i+1, 0, Record[i][0])
        worksheet.write(i+1, 1, Record[i][1])
        worksheet.write(i+1, 2, Record[i][2])
        worksheet.write(i+1, 3, Solutions[Record[i][0]])
    workbook.close()
       
try:
    DataPath = 'class5_测试案例.xlsx'              #数据
    Packages =  OrderedDict()                     #包裹信息
    Warehouse = OrderedDict()                     #仓库信息
    Areas = OrderedDict()                         #区域信息
    Patterns = OrderedDict()                      #模式
    Record = []                                   #数据记录                            
    BatchePattern = {}                            #第一阶段波次-模式对应关系
    Solutions = {}                                #解
    Batches = [100, 120]                          #波次数量
    PackageLimit = [500, 550]                     #单一波次包裹限制
    GoodsLimit = [1800,3000]                      #单一波次商品件数限制
    FirstStageSolveTime = 100                     #求解时间
    SecondStageSolveTime = 100                    #求解时间
    
    starttime = time.time()
    ReadData(DataPath, Packages, Warehouse, Areas, Patterns, Record)
    BuildFirstStageModel(Packages, Warehouse, Patterns, Batches, PackageLimit, GoodsLimit, BatchePattern, FirstStageSolveTime)
    BuildSecondStageModel(Packages, Areas, Patterns, BatchePattern, Solutions, SecondStageSolveTime)
    OutputResult(Solutions, Record)
    endtime = time.time()
    print ("over ! time = ", endtime-starttime)
    

except GurobiError as exception:
    print('Error code ' + str(exception.errno) + ": " + str(exception))

except AttributeError:
    print('Encountered an attribute error')

ReadData !
BuildModel Version 2!
Restricted license - for non-production use only - expires 2022-01-13
Changed value of parameter TimeLimit to 100.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Error code 10010: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license


## 订单配送案例

描述问题,介绍预处理思路以及数学模型,并通过Gurobi获得结果。

在计划期内，企业需要将该周期内所有的订单按照要求配送到客户手中。通常情况下，市场中存在众多的物流供应商，且不通物流供应商零担和整车的规格与费用可能会存在差异，因此如何选择合适的物流供应商使得总配送成本最低是企业需要面对的问题。

从企业的角度，需要考虑如下两个问题，且这两个问题互相影响:
1. 不同订单之间如何组合;
2. 挑选物流供应商以及配送方式(零担或整车)

**处理思路**

- 预处理(不可以拼单的订单)
    - 目的
        - 直接找到成本最低的配送方案, 即确定由哪个物流供应商采取何种方式配送
    - 方法
        - 零担运输, 找出可以用的零担车型, 主要考虑时间、地点和载量的匹配。挑选出成本最低的物流供应商
        - 整车运输, 同理找出可用的整车车型, 主要考虑载量约束时需要满足整车率。从可行方案中挑选出成本最低的配送方案

- 预处理(可以拼单的订单)
    - 目的
        - 以某一订单为标准，识别所有可以和该订单拼单的订单，把这些订单组合成一个集合，在该集合上建立数学模型
    - 方法
        - 对某一特定订单，遍历所有未完成订单，搜索可以平淡的订单。能否平淡取决于起终点和配送时间是否匹配。为了进一步减少模型中的变量数量，对每一个物流供应商，找到所有能够使用的零担和整车车型，不能使用的车型不会出现在变量中。

**拼单订单模型**

- Model Parameters
    - $O$ ~ the set of Orders
    - $P$ ~ the set of logistics providers
    - $V$ ~ the set of types of trucks of a logistics provider
    - $M$ ~ the set of the number of available trucks of a type of a logistics provider
    - $\bar{c}^v_p$ ~ the cost of the v-type truck of logistics provider p
    - $w_i$ ~ the weight of the i-th order
    - $W_p^v$ ~ the capacity of v-type truck of logistics provider p
- Model Variables
    - $x_{ip}^{vm}$ ~ binary variables, equals one if the order i is assigned to the m-th truck of v-type of logistics provider p, otherwise zero
    - $y_p^{vm}$ ~ binary variables, equals one if the m-th truck of v-type of logistics provider p is used, otherwise zero

- obj
    - $min \ \sum_{p,v,m} \bar{c}_p^v x_{ip}^{vm}$
- s.t.
    - all orders have to be delivered
        - $\sum_{p,v,m} x_{ip}^{vm} = 1$, $\forall i \in O$
    - the capacity limitation of the m-th truck which is used
        - $\sum_i w_i x_{ip}^{vm} \le W_p^v y_p^{vm}$, $\forall p \in P, v \in V, m \in M$ 
    - the m-th truck of v-type of logistics provider p is used
        - $\sum_i x_{ip}^{vm} \le M y_p^{vm}$, $\forall p \in P, v \in V, m \in m$ 
    - Variables
        - $x_{ip}^{vm}, y_p^{vm} \in \{0,1\}$, $\forall i \in O, p \in P, v \in V, m \in m$

In [5]:
from gurobipy import *
import xlrd
import xlwt
import sys
from datetime import *
from xlrd import xldate_as_tuple

def ReadData(Orders, Logistics, LogisticsNum):
    data = xlrd.open_workbook("class5_订单信息.xlsx")
    table = data.sheets()[0]
    nrows = table.nrows #行数
    ncols = table.ncols #列数
    for i in range(1,nrows):
        row = []
        for j in range(1,ncols):
            cell = table.cell_value(i,j)
            if table.cell(i,j).ctype == 3:
                date = datetime(*xldate_as_tuple(cell, 0))
                cell = date.strftime('%Y/%m/%d')
                row.append(cell)
            else:
                row.append(cell)
                
        d1 = datetime(int(row[3][0:4]), int(row[3][5:7]), int(row[3][8:]))
        d2 = datetime(int(row[4][0:4]), int(row[4][5:7]), int(row[4][8:]))     
        row.append((d2-d1).days+1)
        row.append(0)
        Orders[table.cell_value(i,0)] = row

    for i in Orders.keys():
        group = [i]
        for j in Orders.keys():
            if i!=j and Orders[i][2] == Orders[j][2] and \
                        Orders[i][3] == Orders[j][3] and \
                        Orders[i][4] == Orders[j][4]:
                group.append(j)
        Orders[i].append(group)
       
    data = xlrd.open_workbook("class5_物流供应商信息.xlsx")
    for i in range(LogisticsNum):
        Logistic = {}
        table = data.sheets()[i]
        nrows = table.nrows #行数
        for j in range(0,nrows):
            row = table.row_values(j) 
            Logistic[row[0]] = row[1:]
        Logistics[i] = Logistic
      
    #print (Logistics)
#挑选报价最便宜的供应商    
def SingleOrder(Orders, index, Logistics, LTL, FTL, FTLRatio):
    print ("Single Order.")
    mincost = 999999
    Vehicleindex = -1000
    LogisticSelect = -1000
    LTLAvailable = -1000     #零担
    FTLAvailable = []        #整车
    
    for i in range(len(LTL)):              #确定零担可用车型
         if Orders[index][1] < LTL[i]:    
             LTLAvailable = i
             break
             
    for i in range(len(FTL)):              #确认整车可用车型
         if  FTL[i]*FTLRatio <= Orders[index][1] and Orders[index][1] <= FTL[i]:
             FTLAvailable.append(i)
 
    if LTLAvailable != -1000 or len(FTLAvailable) != 0:
        for LNum in Logistics.keys():
            if Orders[index][2] in Logistics[LNum].keys() and\
               Orders[index][5] >= Logistics[LNum][Orders[index][2]][7]:   # 确认物流供应商配送该点和时间合理
                if LTLAvailable != -1000:   #确认零担车辆
                    for i in range(LTLAvailable, len(LTL)):
                        if Logistics[LNum][Orders[index][2]][i+1] != '' and\
                           Orders[index][1]*Logistics[LNum][Orders[index][2]][i+1] < mincost:   #报价不为空且低于目前最小报价
                           LogisticSelect = LNum
                           Vehicleindex = i+1
                           mincost = Orders[index][1]*Logistics[LNum][Orders[index][2]][i+1]
                           
                if len(FTLAvailable) != 0:  #确认整车车辆
                    for i in range(len(FTLAvailable)):
                        for j in range(3):
                            if Logistics[LNum][Orders[index][2]][8+FTLAvailable[i]*3+j] != '' and\
                               Logistics[LNum][Orders[index][2]][8+FTLAvailable[i]*3+j] < mincost:   #报价不为空且低于目前最小报价
                               LogisticSelect = LNum
                               Vehicleindex = FTLAvailable[i]*3+8+j
                               mincost = Logistics[LNum][Orders[index][2]][8+FTLAvailable[i]*3+j] 
    
    
    if Vehicleindex != -1000 and LogisticSelect != -1000 and mincost != 99999:   #是否发现安排方法
        Orders[index][6] = 1                      #指示订单是否安排好
        Orders[index].append(LogisticSelect)      #选择的物流供应商
        Orders[index].append(Logistics[LogisticSelect]['南京到'][Vehicleindex])    #具体车型
        if Vehicleindex < 8:
            Orders[index].append('零担')          #零担
        else:
            Orders[index].append('整车')          #整车
        Orders[index].append(mincost)             #配送费用
    else:
        Orders[index][6] = 1
        Orders[index].append('') 
        Orders[index].append('') 
        Orders[index].append('') 
        Orders[index].append('') 
            
def MultiOrders(Orders, index, Logistics, LTL, FTL, FTLRatio):
    print ('Multi-Orders.')
    LTLAvailable = {}  #可用的零担车型
    FTLAvailable = {}  #可用的整车车型
    OrdersOpt = {}
    for i in range(len(Orders[index][7])):
        OrdersOpt [Orders[index][7][i]] = -1
        for j in range(len(LTL)):
             if Orders[Orders[index][7][i]][1] < LTL[j]:
                 LTLAvailable[Orders[index][7][i]] = j
                 break
             
        for j in range(len(FTL)):
             if FTLRatio*FTL[j]<= Orders[Orders[index][7][i]][1] and Orders[Orders[index][7][i]][1] <= FTL[j]:
                 FTLAvailable[Orders[index][7][i]] = j
                 break
             
                
    XINDEX = {}
    PVM = {}
    W = {}   #载重量集合
    for i in LTLAvailable.keys():
        for p in Logistics.keys():
            if Orders[i][2] in Logistics[p].keys() and\
               Orders[i][5] >= Logistics[p][Orders[i][2]][7]:   # 确认物流供应商配送该点和零担时间合理
                   for v in range(LTLAvailable[i], len(LTL)):
                       if Logistics[p][Orders[i][2]][v+1] != '':
                           OrdersOpt[i] = 1
                           for m in range(len(Orders[index][7])):
                               XINDEX[i, p, v+1, m+1] = Orders[i][1]*Logistics[p][Orders[i][2]][v+1]
                               W[i, p, v+1, m+1] = Orders[i][1]
                               PVM[p, v+1, m+1] = LTL[v]
    
    for i in FTLAvailable.keys():
        for p in Logistics.keys():
            if Orders[i][2] in Logistics[p].keys() and\
               Orders[i][5] >= Logistics[p][Orders[i][2]][7]:   # 确认物流供应商配送该点和零担时间合理
                   for v in range(FTLAvailable[i], len(FTL)):
                       for j in range(3):
                           if Logistics[p][Orders[i][2]][8+v*3+j] != '':
                               OrdersOpt[i] = 1
                               for m in range(len(Orders[index][7])):
                                   XINDEX[i, p, 8+v*3+j, m+1] = Logistics[p][Orders[i][2]][8+v*3+j]
                                   W[i, p, 8+v*3+j, m+1] = Orders[i][1]
                                   PVM[p, 8+v*3+j, m+1] = FTL[v]
    modelindex = -1
    for i in range(len(Orders[index][7])):
        if OrdersOpt[Orders[index][7][i]] == -1:    #该订单没有可用供应商
           Orders[Orders[index][7][i]][6] = 1
           Orders[Orders[index][7][i]].append('')
           Orders[Orders[index][7][i]].append('')
           Orders[Orders[index][7][i]].append('')
           Orders[Orders[index][7][i]].append('')
        else:                                       #该订单有可用车型
            modelindex = 1
            
    if modelindex == 1:
        model = Model()
        x = model.addVars(XINDEX.keys(), obj=XINDEX, vtype=GRB.BINARY, name='x')
        y = model.addVars(PVM.keys(), vtype=GRB.BINARY, name='y')
        
        #Constraint 1: all orders have to be delivered.
        for i in OrdersOpt.keys():
            if OrdersOpt[i] == 1:
                model.addConstr(x.sum(i, '*', '*', '*') == 1)
      
        #Constraint 2: the capacity limitation of the m-th truck which is used.
        for i in PVM.keys():
            if i[1] < 8:
                model.addConstr(x.prod(W,'*',i[0], i[1], i[2]) <= PVM[i]*y[i])
            else:
                model.addConstr(x.prod(W,'*',i[0], i[1], i[2]) >= FTLRatio*PVM[i]*y[i])
                model.addConstr(x.prod(W,'*',i[0], i[1], i[2]) <= PVM[i]*y[i])
        
        #Constraint 3: the m-th truck of v-type of logistics provider p is used
        for i in PVM.keys():
            model.addConstr(x.sum('*', i[0], i[1], i[2]) <= len(Orders[index][7])*y[i])
        
        #model.setParam(GRB.Param.LogToConsole, 0)    
        model.setParam(GRB.Param.TimeLimit, 10) 
        model.optimize()  
                  
        for i in XINDEX.keys():
            if x[i].x != 0:
                Orders[i[0]][6] = 1
                Orders[i[0]].append(i[1])
                Orders[i[0]].append(Logistics[p]['南京到'][i[2]])
                if i[2] < 8:
                    Orders[i[0]].append('零担')
                    Orders[i[0]].append(Orders[i[0]][1]*Logistics[i[1]][Orders[i[0]][2]][i[2]])
                else:
                    Orders[i[0]].append('整车') 
                    Orders[i[0]].append(Logistics[i[1]][Orders[i[0]][2]][i[2]])
                
                
                for j in XINDEX.keys():
                    if x[j].x != 0 and i[0] != j[0] and i[1] == j[1] and i[2] == j[2] and i[3] == j[3]:
                        Orders[i[0]].append(j[0]) 

def OutputResult(Orders):
    workbook = xlwt.Workbook()
    worksheet = workbook.add_sheet('class5_订单配送方案')
    worksheet.write(0, 0, label = '订单编号')
    worksheet.write(0, 1, label = 'N/W (T)净重')
    worksheet.write(0, 2, label = 'G/W (T)毛重')
    worksheet.write(0, 3, label = '预计提货日期')
    worksheet.write(0, 4, label = '期望到达日期')
    worksheet.write(0, 5, label = '目的地')
    worksheet.write(0, 6, label = '物流商')
    worksheet.write(0, 7, label = '使用车型')
    worksheet.write(0, 8, label = '运输方式')
    worksheet.write(0, 9, label = '组合订单编号')
    worksheet.write(0, 10, label = '费用')
    
    for i in Orders.keys():
        worksheet.write(int(i), 0, label=i)
        worksheet.write(int(i), 1, label=Orders[i][0])
        worksheet.write(int(i), 2, label=Orders[i][1])
        worksheet.write(int(i), 3, label=Orders[i][3])
        worksheet.write(int(i), 4, label=Orders[i][4])
        worksheet.write(int(i), 5, label=Orders[i][2])
        if Orders[i][8] != '':
            worksheet.write(int(i), 6, label='物流商'+str(int(Orders[i][8])+1))
            worksheet.write(int(i), 7, label=Orders[i][9])
            worksheet.write(int(i), 8, label=Orders[i][10])
            if Orders[i][10]=='整车' and len(Orders[i])>12:
                count = 0
                for j in range(12, len(Orders[i])):
                    if i < Orders[i][j]:
                        count += 1
                if count == len(Orders[i])-12:
                    worksheet.write(int(i), 10, label=Orders[i][11])
                else:
                    worksheet.write(int(i), 10, label=0)                  
            else:
                worksheet.write(int(i), 10, label=Orders[i][11])
                
            if len(Orders[i])>12:
                temp =[] 
                for j in range(12, len(Orders[i])):
                    temp.append(int(Orders[i][j]))
                worksheet.write(int(i), 9, label=str(temp))
            else:
                worksheet.write(int(i), 9, label='')
                    
        else:
            worksheet.write(int(i), 6, label='——')
            worksheet.write(int(i), 7, label='——')
            worksheet.write(int(i), 8, label='——')
            worksheet.write(int(i), 9, label='')
            worksheet.write(int(i), 10, label='')           
    workbook.save('class5_订单配送方案.xls')
try:
    FTLRatio = 0.9
    Orders = {}
    Logistics = {}
    LTL = [0.5, 3, 5, 10, 20, 1000]        #零担载重
    FTL = [2, 5, 10, 15, 20, 25, 30, 32]   #整车车型载重
    ReadData(Orders, Logistics, 2)
    for i in Orders.keys():
        if Orders[i][6] == 0 and len(Orders[i][7]) == 1:
            SingleOrder(Orders, i, Logistics, LTL, FTL, FTLRatio)
        elif Orders[i][6] == 0 and len(Orders[i][7]) != 1:
            MultiOrders(Orders, i, Logistics, LTL, FTL, FTLRatio)              
    OutputResult(Orders)
    print('处理完成。')
    
except GurobiError as exception:
    print('Error code ' + str(exception.errno) + ": " + str(exception))

except AttributeError:
    print('Encountered an attribute error')

Single Order.
Multi-Orders.
Single Order.
Single Order.
Multi-Orders.
Single Order.
Multi-Orders.
Changed value of parameter TimeLimit to 10.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 100 rows, 160 columns and 608 nonzeros
Model fingerprint: 0x7551f234
Variable types: 0 continuous, 160 integer (160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+04, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 80 rows and 96 columns
Presolve time: 0.00s
Presolved: 20 rows, 64 columns, 128 nonzeros
Variable types: 0 continuous, 64 integer (64 binary)
Found heuristic solution: objective 46620.000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | In